# MIT (TA1): From Paper and Code to annotated extraction


## 0. Preprocessing

In [1]:
import requests, os
GPT_KEY = os.environ.get('OPENAI_API_KEY')
API_ROOT = "http://localhost:8000/"

## 1. MIT annotation end to end pipeline

#### We bring every annotation step together: for the original paper text, let's integrate all the extraction modules and output the MIT extraction:
[http://3.83.68.208/#/Paper-2-annotated-vars/upload_file_annotate_annotation_upload_file_extract__post](http://3.83.68.208/#/Paper-2-annotated-vars/upload_file_annotate_annotation_upload_file_extract__post)

In [2]:
scenario_2_paper = "text_s41598-022-06159-x.txt"
with open(scenario_2_paper, 'rb') as f:
    files = {'file': ('filename', f)}
    params = {"gpt_key": GPT_KEY}
    response = requests.post(API_ROOT + "annotation/upload_file_extract",  params=params,  files=files)
    assert response.status_code == 200, response.text
    json_str = response.text
print(json_str)


{"attributes":[{"type":"anchored_entity","amr_element_id":null,"payload":{"id":{"id":"mit0"},"mentions":[{"id":{"id":"mit0"},"name":"ame","extraction_source":null,"provenance":{"method":"MIT extractor V1.0 - text, dataset, formula annotation (chunwei@mit.edu)","timestamp":"2023-10-31T20:46:33.801844"}}],"text_descriptions":[{"id":{"id":"mit0"},"description":"description","grounding":null,"extraction_source":null,"provenance":{"method":"MIT extractor V1.0 - text, dataset, formula annotation (chunwei@mit.edu)","timestamp":"2023-10-31T20:46:33.801844"}},{"id":{"id":"mit0"},"description":"description","grounding":null,"extraction_source":null,"provenance":{"method":"MIT extractor V1.0 - text, dataset, formula annotation (chunwei@mit.edu)","timestamp":"2023-10-31T20:46:33.801844"}}],"value_descriptions":[{"id":{"id":"mit0-value"},"value":{"amount":"numerical value","grounding":null,"extraction_source":null},"units":null,"type":null,"bounds":null,"provenance":{"method":"MIT extractor V1.0 - 

In [3]:
import ta1viewer # SAME REPO AS EXTRACTION VIEWER
from ta1viewer import display_json
#pip install git+https://github.com/orm011/ta1viewer.git@main
display_json(json_str)

In [4]:
import json

json_object = json.dumps(json.loads(json_str), indent=4)
with open("m_temp_scenario_2.json", "w") as f:
    f.write(json_object)

## 2. Interacting with the University of Arizona extraction

#### With both the University of Arizona and MIT extractions, we first build the entity matching mapping for all the extracted variable entities, and then integrate the two integration together with the unified TA1 data model.
[http://3.83.68.208/#/TA1-Integration/upload_files_integration_integration_get_mapping_post](http://3.83.68.208/#/TA1-Integration/upload_files_integration_integration_get_mapping_post)

In [5]:
with open('m_temp_scenario_2.json', 'rb') as f_mit, open('s41598-022-06159.json', 'rb') as f_arizona:
    files = { 'mit_file': ('bucky__mit-extraction_id.json', f_mit, 'application/json'),
        'arizona_file': ('bucky_arizona_output_example.json', f_arizona, 'application/json')}
    params = {"gpt_key": GPT_KEY}
    response = requests.post(API_ROOT + "integration/get_mapping", params=params, files=files)
    assert response.status_code == 200, response.text
    print(response.text)

{"attributes":[{"type":"anchored_entity","amr_element_id":null,"payload":{"id":{"id":"R:-1687473227"},"mentions":[{"id":{"id":"T:1909222834"},"name":"NPI","extraction_source":{"page":1,"block":7,"surrounding_passage":"In Ontario , Canada , the wild type was overtaken first by the Alpha / B1.1.17 variant , and then by the Delta / B.1.617 variant .\nThe principal objective of the present study is to develop and apply a much expanded Susceptible-Infection-Recovered-type model to better understand the spread of multiple VOC , and assess the effectiveness of vaccination and non pharmaceutical interventions ( NPI ) .\nThe model represents competition among VOC , and reveals their mutual inhibitory effects .","char_start":579,"char_end":582,"document_reference":{"id":"s41598-022-06159-x (1).pdf"}},"provenance":{"method":"Skema TR Pipeline rules","timestamp":"2023-10-29T20:32:10.585267"}},{"id":{"id":"mit0"},"name":"ame","extraction_source":null,"provenance":{"method":"MIT extractor V1.0 - tex

In [6]:
display_json(response.text)